In [ ]:
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git@nlp

In [ ]:
import sys
sys.path.append('/kaggle/working')

**[Natural Language Processing Home Page](https://www.kaggle.com/learn/natural-language-processing)**

---


# Vectorizing Language

In this exercise you will use SpaCy to convert the review text into word vectors. 

You'll first get these vectors with SpaCy. Then, you'll use them to train a logistic regression model and a linear support vector classifier.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.nlp.ex3 import *
print("\nSetup complete")

In [ ]:
review_data = pd.read_csv('../input/nlp-course/yelp_ratings.csv', index_col=0)
review_data.head()

In [ ]:
import spacy

# Need to load the large model to get the vectors
nlp = spacy.load('en_core_web_lg')

### Exercise: Get document vectors

Use SpaCy to get document vectors from the review text. 

Returning all 44,500 document vectors takes about 20 minutes, so here you'll need to get only the first 100. For the rest of this exercise, I've provided a file with all 44,500 document vectors.

In [ ]:
# We just want the vectors so we can turn off other models in the pipeline
with nlp.disable_pipes():
    vectors = np.array([nlp(review.text).vector for idx, review in review_data[:100].iterrows()])

The cell below will load in the rest of the document vectors.

In [ ]:
# Loading all document vectors from file
vectors = np.load('../input/nlp-course/review_vectors.npy')

Split the data into train and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vectors, review_data.sentiment, test_size=0.1, random_state=1)

### Exercise: Fit a logistic regression model

Use the document vectors to train a scikit-learn logistic regression model and calculate the accuracy.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

model = LogisticRegression(random_state=1)
model.fit(X_train, y_train)

accuracy = metrics.accuracy_score(y_test, model.predict(X_test))
print(accuracy)

### Exercise: Find the best regularization parameter

You can find a model with better performance by searching through hyperparameter values. For example, here you can adjust the regularization parameter `C`. The easiest way to do this is with cross-validation using `LogisticRegressionCV`. This will automatically split up your data into folds and measure the scoring metric for each fold. Using this, you can search through a range of values for `C` to maximize the accuracy.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
c_vals = [0.1, 1, 10, 100, 1000, 10000]
model = LogisticRegressionCV(Cs=c_vals, scoring='accuracy',
                             cv=5, max_iter=10000,
                             random_state=1).fit(X_train, y_train)

Now you can test the model's performance on the hold-out test data.

In [ ]:
print(f'Model test accuracy: {model.score(X_test, y_test)}')

### Exercise: Try a different model

It's possible to get better accuracy using a different model. Here, try scikit-learn's `LinearSVC` model to see if you can improve on the logisitic regression model. Again, use cross-validation to find the best value for the regularization parameter. This time you'll need to do the cross-validation yourself with `cross_val_score`.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [ ]:
scores, models = [], []
for c in c_vals:
    clf = LinearSVC(C=c, random_state=1, dual=False, max_iter=10000)
    cv_score = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
    scores.append(cv_score.mean())
    models.append(clf)

In [ ]:
max_score = np.array(scores).argmax()
model = models[max_score].fit(X_train, y_train)
print(f'Model test accuracy: {model.score(X_test, y_test)}')

You should get an accuracy of 94.6%, slightly better than the best logistic regression model.

Congratulations on finishing this course! At this point you know how to get embeddings for each word in the documents, but you're only using the averaged document vectors with these models. Using the word vectors themselves might result in even better performing models. To do this you'll want to use a recurrent neural network (RNN for short). We won't cover RNNs in this course, but look them up if you want to learn about state-of-the-art NLP models.

---
**[Natural Language Processing Home Page](https://www.kaggle.com/learn/natural-language-processing)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*